# Position Triangulator
We opted against using a panorama approach. Instead, we compare the robot's latest images against 72 images gathered in the initial phase. Given a potential pose, this compare the recent image to the one of the 72 images closes to that pose. Instead of comparing these images directly, we pretend as if the images aren't perfectly aligned and thus compare several cropped offset versions of each image to the latest image.

## Imports
We use the following packages in our implementation.
- `numpy` arrays allow us to compare the images and get the mean squared error
- `cv2` allows us to read in images from files
- `math` provides important constants and functions for our error calculations
- `random` allows random particle generation
- `cozmo` allows us to communicate with the robot. `cozmo.util` helps us to tell the robot some umber of degrees
- `matplotlib.pyplot` is used to show the current beliefs of the robot in a histogram
- `os` is used for joining file paths
- `sys` gives us an integer maxsize as our starting point when looking at differences between images

In [1]:
import numpy as np
import cv2
import math
import random
import cozmo
from cozmo.util import degrees
import matplotlib.pyplot as plt
import os
import sys

In [2]:
degree_increment = 5

In [3]:
parent_dir = "/Users/ironi/Documents/Conda/Cozmo"
path = os.path.join(parent_dir, "Images")

In [4]:
def take_pictures(robot: cozmo.robot.Robot):
  # Initial setup, generate repeatable head angle and lift height
  robot.say_text("Taking pictures").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE - cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()

  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True

  deg_step = 5

  # Dictionary to store the first occurrence and size of each cube
  cube_info = {} 
  
  # Saves picture of what Cozmo sees every 10 degrees.
  for d in range(0, 360, deg_step):
    # fileName = "image" + str(d) + ".jpeg"
 
    robot.turn_in_place(degrees(deg_step)).wait_for_completed()
    
    # latest_image = robot.world.latest_image
    # # annotation seems to be important, we have had issues saving images if we don't annotate
    # annotated = latest_image.annotate_image()

    cube_id = -1

    objects = robot.world.visible_objects
    for obj in objects:
        if('LightCube' in obj.descriptive_name):
          size = (obj.last_observed_image_box.width, obj.last_observed_image_box.height)
          size = size[0]*size[1]
          # print("Hello")
          # print(obj.descriptive_name[10])
          cube_id = int(obj.descriptive_name[10])
          continue

    if cube_id > 0:
                
      # Check if this is the first occurrence of the cube
      if cube_id not in cube_info:
                    
        # Store first occurrence and size of the cube
        cube_info[cube_id] = {
            'first_occurrence': d,
            'size': size
        }

    # if latest_image is not None:
    #   # print("image = %s" % latest_image)
    #   converted = annotated.convert()
    #   file_path = os.path.join(path, fileName)
    #   converted.save(file_path, "JPEG", resolution=10)
    
  print(cube_info)
  return cube_info



In [5]:
def return_to_start(robot: cozmo.robot.Robot):
    # Initial setup, generate repeatable head angle and lift height
  robot.say_text("Taking pictures").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE - cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()

  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True

  deg_step = 5

  # Dictionary to store the first occurrence and size of each cube
  new_cube_info = {} 
  
  # Saves picture of what Cozmo sees every 10 degrees.
  for d in range(0, 360, deg_step):
    # fileName = "image" + str(d) + ".jpeg"
 
    robot.turn_in_place(degrees(deg_step)).wait_for_completed()
    
    # latest_image = robot.world.latest_image
    # # annotation seems to be important, we have had issues saving images if we don't annotate
    # annotated = latest_image.annotate_image()

    cube_id = -1

    objects = robot.world.visible_objects
    if objects:
      for obj in objects:
        if('LightCube' in obj.descriptive_name):
          size = (obj.last_observed_image_box.width, obj.last_observed_image_box.height)
          size = size[0]*size[1]
          # print("Hello")
          # print(obj.descriptive_name[10])
          cube_id = int(obj.descriptive_name[10])
          continue

    if cube_id > 0:
                
      # Check if this is the first occurrence of the cube
      if cube_id not in new_cube_info:
                    
        # Store first occurrence and size of the cube
        new_cube_info[cube_id] = {
            'first_occurrence': d,
            'size': size
        }

    # if latest_image is not None:
    #   # print("image = %s" % latest_image)
    #   converted = annotated.convert()
    #   file_path = os.path.join(path, fileName)
    #   converted.save(file_path, "JPEG", resolution=10)
    
  print(new_cube_info)

In [6]:
cube_info = cozmo.run_program(take_pictures)

2024-05-02 00:04:27,105 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2024-05-02 00:04:27,108 cozmo.general INFO     Found robot id=1
2024-05-02 00:04:27,113 cozmo.general INFO     Connected to iOS device_id=27 serial=00008110-000E71362EC2401E
2024-05-02 00:04:27,271 cozmo.general INFO     Robot id=1 serial=45a2a81d initialized OK
2024-05-02 00:04:27,292 cozmo.general INFO     Robot delocalized - invalidating poses for all objects
2024-05-02 00:05:26,931 cozmo.general INFO     Shutting down connection
2024-05-02 00:05:26,932 cozmo.general INFO     iOS device_id=27 disconnected.


{3: {'first_occurrence': 30, 'size': 11066.206319220364}, 1: {'first_occurrence': 215, 'size': 8684.802240167744}, 2: {'first_occurrence': 325, 'size': 34921.43059352785}}


In [7]:
input("Press enter after randomly rotating the robot...")

''

In [8]:
cozmo.run_program(return_to_start)